### Unsupervised Learning : Cluster Assignment

#### Paper: Alternatives to the k-means algorithm that find better clusterings

#### Author: Joaquim Marset Alsina

### Imports

In [ ]:
from datasets.birch_data import generate_birch_data
from datasets.pelleg_moore_data import generate_pelleg_moore_data
from datasets.image_data import *
from datasets.adult_data import AdultData

from experiments import experiment_1, experiment_2, experiment_3, experiment_4
from utils.cluster_initialization import *


from utils.plot_utils import *
from utils.string_utils import *
from utils.metrics import evaluate_resulting_clusters

import os
import numpy as np
import time
from sklearn.decomposition import PCA

### Create required folders

In [ ]:
root_path = './'

datasets_path = os.path.join(root_path, 'datasets')
images_path = os.path.join(datasets_path, 'images')

results_path = os.path.join(root_path, 'results')
experiment_1_results_path = os.path.join(results_path, 'experiment_1')
experiment_2_results_path = os.path.join(results_path, 'experiment_2')
experiment_3_results_path = os.path.join(results_path, 'experiment_3')
experiment_4_results_path = os.path.join(results_path, 'experiment_4')

In [ ]:
os.makedirs(results_path, exist_ok=True)
os.makedirs(experiment_1_results_path, exist_ok=True)
os.makedirs(experiment_2_results_path, exist_ok=True)
os.makedirs(experiment_3_results_path, exist_ok=True)
os.makedirs(experiment_4_results_path, exist_ok=True)

### Experiment 1: Convergence properties of the 5 described algorithms on the BIRCH synthetic data

In [ ]:
# This first experiment belongs to the paper. The authors wanted to test the convergence of the 5 described algorithms using a synthetic data they extracted 
# from the BIRCH paper

grid_size = 10
num_clusters = grid_size * grid_size
points_per_cluster = 100
center_distance = 4 * np.sqrt(2)

birch_data, true_clustering, true_centers = generate_birch_data(grid_size, center_distance, points_per_cluster)

cluster_radius = np.linalg.norm(true_centers[1] - true_centers[0]) / 4

plot_generated_synthetic_data(birch_data, true_centers, true_clustering, BIRCH, experiment_1_results_path, points_per_cluster)

# They considered to perform only 100 iterations and 1 repetition of the execution, and compute the number of true clusterings each algorithm was able to find
# as well as compute the quality of the predicted clustering. This quality was computed as the squared root of the K-Means objective as a way to unify the comparison
# even though each algorithm minimizes its own objective function

# They repeated the experiment 2 times with different cluster initializations: Forgy and Random partition

repetitions = 1
iterations = 500

# Before running the algorithms, we are going to plot the initial centers to see how each initialization method works

initial_centers_forgy = initialize_centers_forgy(num_clusters, birch_data)
initial_centers_rand_part = initialzie_centers_random_partition(num_clusters, birch_data)

plot_initial_cluster_centers(birch_data, initial_centers_forgy, BIRCH, FORGY, experiment_1_results_path)

plot_initial_cluster_centers(birch_data, initial_centers_rand_part, BIRCH, RANDPART, experiment_1_results_path)

#### Forgy initialization

In [ ]:
kmeans_function = experiment_1.run_kmeans
fuzzy_cmeans_function = experiment_1.run_fuzzy_c_means
kharmonic_function = experiment_1.run_kharmonic
hybrid_1_function = experiment_1.run_hybrid_1
hybrid_2_function = experiment_1.run_hybrid_2
gem_function = experiment_1.run_gaussian_em

# We define some parameters as a dictionary as we have tried to create a unified framework to run the experiment
kmeans_params = {
    'iterations' : iterations,
    'repetitions' : repetitions
}

fuzzy_cmeans_params = {
    'fuzzy_degree' : 1.3,
    'convergence_threshold' : 0.0001,
    'iterations' : iterations,
    'repetitions' : repetitions
}

kharmonic_params = {
    'p' : 3.5,
    'convergence_threshold' : 0.0001,
    'iterations' : iterations,
    'repetitions' : repetitions
}

gem_params = {
    'iterations': iterations, 
    'repetitions': repetitions,
    'cov_init_value': 0.2, 
    'convergence_threshold': 0.0001
}

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_forgy, FORGY, KMEANS, 
    experiment_1_results_path, kmeans_function, kmeans_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_forgy, FORGY, FCMEANS, 
    experiment_1_results_path, fuzzy_cmeans_function, fuzzy_cmeans_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_forgy, FORGY, KHARMONIC, 
    experiment_1_results_path, kharmonic_function, kharmonic_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_forgy, FORGY, HYBRID1, 
    experiment_1_results_path, hybrid_1_function, kharmonic_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_forgy, FORGY, HYBRID2, 
    experiment_1_results_path, hybrid_2_function, kharmonic_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_forgy, 
    FORGY, GEM, experiment_1_results_path, gem_function, gem_params)

#### Random partition initialization

In [ ]:
# We repeat the process with the other cluster initialization

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_rand_part, 
    RANDPART, KMEANS, experiment_1_results_path, kmeans_function, kmeans_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_rand_part, 
    RANDPART, FCMEANS, experiment_1_results_path, fuzzy_cmeans_function, fuzzy_cmeans_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_rand_part, 
    RANDPART, KHARMONIC, experiment_1_results_path, kharmonic_function, kharmonic_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_rand_part, 
    RANDPART, HYBRID1, experiment_1_results_path, hybrid_1_function, kharmonic_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_rand_part, 
    RANDPART, HYBRID2, experiment_1_results_path, hybrid_2_function, kharmonic_params)

experiment_1.run_algorithm_with_birch_data(birch_data, num_clusters, initial_centers_rand_part, 
    RANDPART, GEM, experiment_1_results_path, gem_function, gem_params)

### Experiment 2: Paper's second experiment

In [ ]:
# In this experiment the authors wanted to see the average-case behaviour of the described algorithms. 
# This is why they created multiple synthetic algorithms based on a particular paper (which they called Pelleg and Moore data honouring the authors)
# They wanted to test different number of dimensions for the data, creating multiple datasets for each dimension. In particular, 
# they created data with 2, 4, and 6 dimensions, creating 100 datasets for each. Each dataset contains 50 clusters and a total of 2500 points
# As before, they repeated the experiment with both initializations

dimensions_list = [2, 4, 6]
num_datasets = 100
num_clusters = 50
points_per_cluster = 50

In [ ]:
# The clusters centers are generated random by sampling in the unit hypercube, and the points are generated according to a Gaussian
# centered in the true center with a standard deviation of num_dimensions*0.012

std_factor = 0.012

# Each algorithm runs for 100 iterations, all sharing the same centers
iterations = 100

# To compute the quality metric we need to run a k-means with the real centers and let it converge. We are assuming they are referring 
# to the real centers because the paper says "running KM to convergence starting with the centers that generated the data sets"
# Therefore, it will probably last a few iterations, but the objective function at convergence will not change
optimal_kmeans_iterations = 500

# We are assuming as the previous experiment only one repetition of each algorithm with each initialization method
repetitions = 1

# We define the other algorithm parameters as the authors did
fuzzy_degree = 1.3
p = 3.5
convergence_threshold = 0.001
cov_init_value = 0.2

In [ ]:
# Plot an example of dataset with 2 dimensions
pelleg_data, true_clustering, true_centers = generate_pelleg_moore_data(2, num_clusters, points_per_cluster, std_factor)
plot_generated_synthetic_data(pelleg_data, true_centers, true_clustering, PELLEG, experiment_2_results_path)

In [ ]:
kmeans_function = experiment_2.run_kmeans
fuzzy_function = experiment_2.run_fuzzy_c_means
kharmonic_function = experiment_2.run_kharmonic
hybrid_1_function = experiment_2.run_hybrid_1
hybrid_2_function = experiment_2.run_hybrid_2
gaussian_em_function = experiment_2.run_gaussian_em

kmeans_params = {'iterations': iterations, 'repetitions': repetitions}
fuzzy_params = {'iterations': iterations, 'repetitions': repetitions, 'fuzzy_degree': fuzzy_degree, 'convergence_threshold': convergence_threshold}
kharmonic_params = {'iterations': iterations, 'repetitions': repetitions, 'p': p, 'convergence_threshold': convergence_threshold}
gaussian_em_params = {'iterations': iterations, 'repetitions': repetitions, 'cov_init_value': cov_init_value, 'convergence_threshold': convergence_threshold}

functions = [kmeans_function, fuzzy_function, kharmonic_function, hybrid_1_function, hybrid_2_function]
params = [kmeans_params, fuzzy_params, kharmonic_params, kharmonic_params, kharmonic_params, gaussian_em_params]
names = [KMEANS, FCMEANS, KHARMONIC, HYBRID1, HYBRID2]

In [ ]:
def run_experiment_2_initialization(init_method, init_name):
    num_methods = len(functions)
    
    for dimensions in dimensions_list:

        start_time = time.time()

        all_dataset_ratios = np.zeros((num_datasets, num_methods, iterations))

        for i in range(num_datasets):

            data, _, true_centers = generate_pelleg_moore_data(dimensions, num_clusters, points_per_cluster, std_factor)
            optimal_quality_values = experiment_2.run_kmeans(data, num_clusters, true_centers, optimal_kmeans_iterations, 1)

            initial_centers = init_method(num_clusters, data)

            ratios = experiment_2.run_experiment_dimensions_dataset(data, num_clusters, optimal_quality_values, initial_centers, functions, params)
            all_dataset_ratios[i] = ratios
        
        avg_ratios = np.mean(all_dataset_ratios, axis=0)
        plot_algorithms_ratio_quality_comparison(avg_ratios, names, PELLEG, init_name, dimensions, num_datasets, './')

        print(f'Time to run {num_datasets} with {dimensions} dimensions: {time.time() - start_time:.2f} seconds')

In [ ]:
run_experiment_2_initialization(initialize_centers_forgy, FORGY)
run_experiment_2_initialization(initialzie_centers_random_partition, RANDPART)

### Experiment 3: Semantic segmentation

In [ ]:
# This is one of the paper's experiment where the authors wanted to compare the performance of the K-Means and the K-Harmonic Means when performing 
# semantic segmentation of a particular image of a hand. They assesses the quality both visually and with the K-Means quality metric

image_path = os.path.join(images_path, 'hand.png')
image = load_image(image_path, (100, 100))
scaler, image_data = generate_image_data(image)
image_name = 'hand'

# In the paper they say they used 5 clusters, but they did not include any of the other parameters. We are going to set them as follow:
num_clusters = 5
iterations = 500
repetitions = 1

# They also repeated the experiment with both Forgy and Random partition initialization of the clusters
initial_centers_forgy = initialize_centers_forgy(num_clusters, image_data)
initial_centers_rand_part = initialzie_centers_random_partition(num_clusters, image_data)


#### Forgy

In [ ]:
k_means_function = experiment_3.run_kmeans
kharmonic_function = experiment_3.run_kharmonic

k_means_params = {
        'iterations' : iterations,
        'repetitions' : repetitions
}

kharmonic_params = {
        'p' : 2,
        'convergence_threshold' : 0.0001,
        'iterations' : iterations,
        'repetitions' : repetitions
}

experiment_3.run_semantic_segmentation(image, image_data, scaler, num_clusters, initial_centers_forgy, FORGY, KMEANS, 
        image_name, experiment_3_results_path, k_means_function, k_means_params)

experiment_3.run_semantic_segmentation(image, image_data, scaler, num_clusters, initial_centers_forgy, FORGY, KHARMONIC, image_name,
        experiment_3_results_path, kharmonic_function, kharmonic_params)

#### Random partition

In [ ]:
experiment_3.run_semantic_segmentation(image, image_data, scaler, num_clusters, initial_centers_rand_part, RANDPART, KMEANS, image_name,
        experiment_3_results_path, k_means_function, k_means_params)

experiment_3.run_semantic_segmentation(image, image_data, scaler, num_clusters, initial_centers_rand_part, RANDPART, KHARMONIC, image_name,
        experiment_3_results_path, kharmonic_function, kharmonic_params)

#### Repeat the process increasing the number of clusters

In [ ]:
num_clusters = 10
image_name = 'hand_10_clusters'

initial_centers_forgy = initialize_centers_forgy(num_clusters, image_data)
initial_centers_rand_part = initialzie_centers_random_partition(num_clusters, image_data)

experiment_3.run_semantic_segmentation(image, image_data, scaler, num_clusters, initial_centers_forgy, FORGY, KMEANS, image_name,
        experiment_3_results_path, k_means_function, k_means_params)

experiment_3.run_semantic_segmentation(image, image_data, scaler, num_clusters, initial_centers_forgy, FORGY, KHARMONIC, image_name,
        experiment_3_results_path, kharmonic_function, kharmonic_params)

experiment_3.run_semantic_segmentation(image, image_data, scaler, num_clusters, initial_centers_rand_part, RANDPART, KMEANS, image_name,
        experiment_3_results_path, k_means_function, k_means_params)

experiment_3.run_semantic_segmentation(image, image_data, scaler, num_clusters, initial_centers_rand_part, RANDPART, KHARMONIC, image_name,
        experiment_3_results_path, kharmonic_function, kharmonic_params)

### Experiment 4: Compare K-Harmonic Means with some Sklearn algorithms

#### Adult dataset

In [ ]:
# Read and preprocess the data from the arff file, obtaining the true classification label (i.e. true cluster) and the feature names
adult_arff_path = os.path.join(datasets_path, 'adult.arff')
adult_object = AdultData(adult_arff_path, experiment_4_results_path)
data, true_clustering, feature_names = adult_object.get_content()

In [ ]:
# Apply PCA to determine the number of principal components that account for the 90% of the data variance
pca_object = PCA()
pca_object.fit(data)
plot_pca_explained_variance(pca_object.explained_variance_ratio_, ADULT, experiment_4_results_path)

In [ ]:
# Looking at the generated plot, we see that we need the first 17 principal components
number_of_pcs = 17
pca_object = PCA(number_of_pcs)
reduced_data = pca_object.fit_transform(data)

# Let's plot the true clustering with the reduced data
pca_feature_names = ['PC1', 'PC2', 'PC3']
plot_clustering_3d(reduced_data, true_clustering, pca_feature_names, ADULT, 'Ground-truth', experiment_4_results_path)

In [ ]:
# Let's now run different algorithms, so we are going to define some common parameters to all the algorithms

# List with the numbers of clusters to try
k_values = range(2, 16)
# Number of max iterations to run each algorithm
iterations = 500
# Number of repetitions with different initial centers
repetitions = 15

##### Sklearn K-Means

In [ ]:
# Let's first search for the optimal number of clusters using the elbow method
# The true number of clusters is 2, so let's try a range between 2 and 15

kmeans_function = experiment_4.run_sklearn_kmeans
# We define some parameters as a dictionary as we have tried to create a unified framework to run the experiments
kmeans_params = {
    'iterations' : iterations,
    'repetitions' : repetitions
}

experiment_4.search_optimal_number_of_clusters(reduced_data, k_values, SKLEARN_KMEANS, ADULT, experiment_4_results_path, kmeans_function, kmeans_params)

In [ ]:
# Looking at the plot for the silhouette score to apply the elbow method (inverse in this case), we see that the optimal number of clusters is 4 (the point where is maximum)
optimal_k_skmeans = 4

# Let's now apply the algorithm again with this number of clusters
skmeans_clustering, skmeans_centers = experiment_4.run_algorithm_with_optimal_k(reduced_data, optimal_k_skmeans, kmeans_function, kmeans_params)

# Let's now compute some evaluation metrics, plot the resulting clustering, and save the predicted centers
evaluate_resulting_clusters(reduced_data, true_clustering, skmeans_clustering, ADULT, SKLEARN_KMEANS, experiment_4_results_path)
plot_clustering_3d(reduced_data, skmeans_clustering, pca_feature_names, ADULT, SKLEARN_KMEANS, experiment_4_results_path)

skmeans_centers = pca_object.inverse_transform(skmeans_centers)
skmeans_centers = adult_object.restore_center_features(skmeans_centers)
write_cluster_centers_to_txt(skmeans_centers, feature_names, SKLEARN_KMEANS, ADULT, experiment_4_results_path)

##### K-Harmonic Means

In [ ]:
# Let's repeat all the process with the K-Harmonic Means algorithm (the one studied in the paper)
kharmonic_function = experiment_4.run_kharmonic
kharmonic_params = {
    'p' : 2,
    'convergence_threshold' : 0.001,
    'iterations' : iterations,
    'repetitions' : repetitions
}

#experiment_4.search_optimal_number_of_clusters(reduced_data, k_values, KHARMONIC, ADULT, experiment_4_results_path, kharmonic_function, kharmonic_params)

In [ ]:
# Looking at the plot now we obtain 2 as the optimal number of clusters (the same as in the original data)
optimal_k_kharmonic = 2

kharmonic_clustering, kharmonic_centers = experiment_4.run_algorithm_with_optimal_k(reduced_data, optimal_k_kharmonic, kharmonic_function, kharmonic_params)

evaluate_resulting_clusters(reduced_data, true_clustering, kharmonic_clustering, ADULT, KHARMONIC, experiment_4_results_path)
plot_clustering_3d(reduced_data, kharmonic_clustering, pca_feature_names, ADULT, KHARMONIC, experiment_4_results_path)

kharmonic_centers = pca_object.inverse_transform(kharmonic_centers)
kharmonic_centers = adult_object.restore_center_features(kharmonic_centers)
write_cluster_centers_to_txt(kharmonic_centers, feature_names, KHARMONIC, ADULT, experiment_4_results_path)

##### Fuzzy C-Means

In [ ]:
fcmeans_function = experiment_4.run_fuzzy_c_means
fcmeans_params = {
    'fuzzy_degree' : 2,
    'convergence_threshold' : 0.001,
    'iterations' : iterations,
    'repetitions' : repetitions
}

experiment_4.search_optimal_number_of_clusters(reduced_data, k_values, FCMEANS, ADULT, experiment_4_results_path, fcmeans_function, fcmeans_params)

In [ ]:
optimal_k_fuzzy = 2

fcmeans_clustering, fcmeans_centers = experiment_4.run_algorithm_with_optimal_k(reduced_data, optimal_k_fuzzy, fcmeans_function, fcmeans_params)

evaluate_resulting_clusters(reduced_data, true_clustering, fcmeans_clustering, ADULT, FCMEANS, experiment_4_results_path)
plot_clustering_3d(reduced_data, fcmeans_clustering, pca_feature_names, ADULT, FCMEANS, experiment_4_results_path)

fcmeans_centers = pca_object.inverse_transform(fcmeans_centers)
fcmeans_centers = adult_object.restore_center_features(fcmeans_centers)
write_cluster_centers_to_txt(fcmeans_centers, feature_names, FCMEANS, ADULT, experiment_4_results_path)

##### Hybrid 1

In [ ]:
hybrid1_function = experiment_4.run_hybrid_1

experiment_4.search_optimal_number_of_clusters(reduced_data, k_values, HYBRID1, ADULT, experiment_4_results_path, hybrid1_function, kharmonic_params)

In [ ]:
optimal_k_h1 = 7

h1_clustering, h1_centers = experiment_4.run_algorithm_with_optimal_k(reduced_data, optimal_k_h1, hybrid1_function, kharmonic_params)

evaluate_resulting_clusters(reduced_data, true_clustering, h1_clustering, ADULT, HYBRID1, experiment_4_results_path)
plot_clustering_3d(reduced_data, h1_clustering, pca_feature_names, ADULT, HYBRID1, experiment_4_results_path)

h1_centers = pca_object.inverse_transform(h1_centers)
h1_centers = adult_object.restore_center_features(h1_centers)
write_cluster_centers_to_txt(h1_centers, feature_names, HYBRID1, ADULT, experiment_4_results_path)

##### Hybrid 2

In [ ]:
hybrid2_function = experiment_4.run_hybrid_2

experiment_4.search_optimal_number_of_clusters(reduced_data, k_values, HYBRID2, ADULT, experiment_4_results_path, hybrid2_function, kharmonic_params)

In [ ]:
optimal_k_h2 = 2

h2_clustering, h2_centers = experiment_4.run_algorithm_with_optimal_k(reduced_data, optimal_k_h2, hybrid2_function, kharmonic_params)

evaluate_resulting_clusters(reduced_data, true_clustering, h2_clustering, ADULT, HYBRID2, experiment_4_results_path)
plot_clustering_3d(reduced_data, h2_clustering, pca_feature_names, ADULT, HYBRID2, experiment_4_results_path)

h2_centers = pca_object.inverse_transform(h2_centers)
h2_centers = adult_object.restore_center_features(h2_centers)
write_cluster_centers_to_txt(h2_centers, feature_names, HYBRID2, ADULT, experiment_4_results_path)

#### Sklearn Gaussian EM

In [ ]:
gaussian_EM_function = experiment_4.run_sklearn_gaussian_EM
gaussian_EM_params = {
    'convergence_threshold' : 0.001,
    'iterations' : iterations,
    'repetitions' : repetitions
}

#experiment_4.search_optimal_number_of_clusters(reduced_data, k_values, SKLEARN_GEM, ADULT, experiment_4_results_path, gaussian_EM_function, gaussian_EM_params)

In [ ]:
optimal_k_gaussian_EM = 2

gaussian_EM_clustering, gaussian_EM_centers = experiment_4.run_algorithm_with_optimal_k(reduced_data, optimal_k_gaussian_EM, gaussian_EM_function, gaussian_EM_params)

evaluate_resulting_clusters(reduced_data, true_clustering, gaussian_EM_clustering, ADULT, SKLEARN_GEM, experiment_4_results_path)
plot_clustering_3d(reduced_data, gaussian_EM_clustering, pca_feature_names, ADULT, SKLEARN_GEM, experiment_4_results_path)

gaussian_EM_centers = pca_object.inverse_transform(gaussian_EM_centers)
gaussian_EM_centers = adult_object.restore_center_features(gaussian_EM_centers)
write_cluster_centers_to_txt(gaussian_EM_centers, feature_names, SKLEARN_GEM, ADULT, experiment_4_results_path)